# Clustering performance when $k$ must be estimated

In [ ]:
from sklearn.metrics import accuracy_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from tqdm import tqdm
import time
import json

from models.baseline_model import ArticleBaselineModel
from models.tabularncd_model import TabularNCDModel
from models.NCD_Spectral_Clustering import *
from models.NCD_Kmeans import k_means_pp
from models.PBN_model import PBNModel
from src.dataset_utils import *
from src.utils import *

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe'  # So we can view the plots in the jupyter notebook

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = setup_device(use_cuda=True)

## Table of contents:
* [1. Define some functions](#1)
* [2. $k$ estimation in the original feature space](#2)
* [2. Clustering models](#2)
    * [2.1 $k$-means](#2.1)
    * [2.2 NCD $k$-means](#2.2)
    * [2.3 Spectral Clustering](#2.3)
    * [2.4 NCD Spectral Clustering](#2.4)
* [3. NCD models](#3)
    * [3.1 Baseline](#3.1)
    * [3.2 PBN](#3.2)
    * [3.3 TabularNCD](#32.3)

### Import the dataset

In [ ]:
dataset_names = ['HumanActivityRecognition', 'LetterRecognition', 'Pendigits', 'USCensus1990', 'multiple_feature', 'optdigits', 'cnae_9']

dataset_name = 'HumanActivityRecognition'

x_train, y_train, x_test, y_test, unknown_class_value, y_train_save, y_test_save = import_dataset_with_name(dataset_name)

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float, device=device)
x_test = torch.tensor(x_test, dtype=torch.float, device=device)

# For plots
y_train_unknown_save = y_train_save[y_train == unknown_class_value]
y_test_unknown_save = y_test_save[y_test == unknown_class_value]

# For evaluation
y_train_unknown_save_codes = np.array(pd.Series(y_train_unknown_save).astype('category').cat.codes)
y_test_unknown_save_codes = np.array(pd.Series(y_test_unknown_save).astype('category').cat.codes)

In [ ]:
plot_classes_distribution(y_train_save[y_train!=unknown_class_value], y_train_save[y_train==unknown_class_value], y_test_save, dataset_name)

In [ ]:
x_train_known = x_train[y_train != unknown_class_value]
x_train_unknown = x_train[y_train == unknown_class_value]
y_train_known = y_train[y_train != unknown_class_value]

x_test_known = x_test[y_test != unknown_class_value]
x_test_unknown = x_test[y_test == unknown_class_value]
y_test_known = y_test[y_test != unknown_class_value]

# We need the targets to be in {0, ..., C^l} exactly
classifier_mapper, classifier_ind = np.unique(y_train_known, return_inverse=True)
classifier_mapping_dict = dict(zip(y_train_known, classifier_ind))

y_train_known = np.array(list(map(classifier_mapping_dict.get, y_train_known)))
y_test_known = np.array(list(map(classifier_mapping_dict.get, y_test_known)))

gt_k = len(np.unique(y_test_unknown_save_codes))  # Only used to print errors

# 1) Define some functions <a class="anchor" id="1"></a>

In [ ]:
def get_k_means_predictions(x, k_range, avg=5):
    """
    return: scores : shape=(len(range), avg)
    """
    predictions = []
    for k_clust in tqdm(k_range):
        tmp_predictions = []
        for _ in np.arange(avg):
            km = KMeans(n_clusters=k_clust, init='k-means++', n_init=10)
            y_pred = km.fit_predict(x.cpu().numpy())
            tmp_predictions.append(y_pred)
        predictions.append(tmp_predictions)
    return np.array(predictions)

In [ ]:
def get_silhouette_curve(x, preds):
    scores = []
    for i in range(preds.shape[0]):
        tmp_scores = []
        for j in range(preds.shape[1]):
            tmp_scores.append(silhouette_score(x, preds[i, j]))
        scores.append(tmp_scores)
    return np.array(scores)

In [ ]:
def estimate_k_with_silhouette(k_min, k_max, x):
    k_range = np.arange(k_min, k_max + 1, 1)

    predictions = get_k_means_predictions(x, k_range=k_range, avg=3)

    silhouette_scores = get_silhouette_curve(x.cpu().numpy(), predictions)
    
    sil_k_estimation = k_range[np.argmax(silhouette_scores.mean(axis=1))]

    return sil_k_estimation, silhouette_scores

In [ ]:
def plot_k_means_perf_curve(metric, range, n_classes, metric_name="Silhouette"):
    fig = make_subplots(rows=1, cols=1, horizontal_spacing=0.05, vertical_spacing=0.1)
    
    fig.add_trace(go.Scatter(x=range, y=metric.mean(axis=1),
                             error_y=dict(type='data', array=metric.std(axis=1), visible=True),
                             mode="lines+markers", name=metric_name), row=1, col=1, secondary_y=False)
    
    best_perf = range[np.argmax(metric.mean(axis=1))]
    pos = "top right" if best_perf < n_classes else "top left"
    fig.add_vline(x=n_classes, annotation_text=f"Ground truth: {n_classes}", annotation_position=pos, line_color='red', annotation_font_color='red', secondary_y=False)
    pos = "top left" if best_perf < n_classes else "top right"
    fig.add_vline(x=best_perf, annotation_text=f"Estimation: {best_perf}", annotation_position=pos, line_color='green', annotation_font_color='green', secondary_y=False)
    
    fig.update_xaxes(title="k clusters")
    fig.update_yaxes(title=metric_name)
    fig.update_layout(height=400, width=700, showlegend=False)  # , title=dataset_name
    fig.show()

# 2) $k$ estimation in the original feature space <a class="anchor" id="2"></a>

In [ ]:
k_est, silhouette_scores = estimate_k_with_silhouette(k_min=2, k_max=30, x=x_train_unknown)

plot_k_means_perf_curve(silhouette_scores, np.arange(2, 30, 1), gt_k)

print(f"Estimated k={k_est}, ground truth={gt_k} (difference={np.abs(gt_k - k_est)})")

# 3) Clustering models <a class="anchor" id="3"></a>

## 3.1 $k$-means <a class="anchor" id="3.1"></a>

In [ ]:
accs_1, nmis_1, aris_1 = [], [], []

for i in tqdm(range(10)):
    km = KMeans(n_clusters=k_est, init='random', n_init=10).fit(x_train_unknown.cpu().numpy())
    
    y_test_unknown_pred = km.predict(x_test_unknown.cpu().numpy())
    accs_1.append(hungarian_accuracy(y_test_unknown_save_codes, y_test_unknown_pred))
    nmis_1.append(normalized_mutual_info_score(y_test_unknown_save_codes, y_test_unknown_pred))
    aris_1.append(adjusted_rand_score(y_test_unknown_save_codes, y_test_unknown_pred))

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_1)*100, np.std(accs_1)*100, np.mean(nmis_1)*100, np.std(nmis_1)*100, np.mean(aris_1)*100, np.std(aris_1)*100))

## 3.2 NCD $k$-means <a class="anchor" id="3.2"></a>

In [ ]:
# For this method, we define the centroids of the known classes with ground truth as initial centroids
known_classes_centroids = torch.stack([x_train_known[y_train_known==c].mean(axis=0) for c in np.unique(y_train_known)])

centroid_to_class_dict = dict(enumerate(np.unique(y_train[y_train != unknown_class_value])))

In [ ]:
accs_2, nmis_2, aris_2 = [], [], []

for i in tqdm(range(10)):
    kmpp = k_means_pp(pre_centroids=known_classes_centroids, k_new_centroids=k_est)
    kmpp.fit(x_train_unknown, tolerance=1e-4, n_iterations=300)
    
    y_test_unknown_pred = kmpp.predict_unknown_data(x_test_unknown).cpu().numpy()
    accs_2.append(hungarian_accuracy(y_test_unknown_save_codes, y_test_unknown_pred))
    nmis_2.append(normalized_mutual_info_score(y_test_unknown_save_codes, y_test_unknown_pred))
    aris_2.append(adjusted_rand_score(y_test_unknown_save_codes, y_test_unknown_pred))

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_2)*100, np.std(accs_2)*100, np.mean(nmis_2)*100, np.std(nmis_2)*100, np.mean(aris_2)*100, np.std(aris_2)*100))

## 3.3 Spectral Clustering <a class="anchor" id="3.3"></a>

#### /!\ We feed our custom adjacency matrix to sklearn's Spectral Clustering /!\
Instead of a sigma, we use a custom parameter, min_dist.
See the paper for more details

In [ ]:
sigma = estimate_sigma(batch_outer_difference_normed(x_train_unknown), 0.001)
print("Using min_dist = 0.001, estimated a sigma =", sigma)

test_outer_diff = batch_outer_difference_normed(x_test_unknown)
test_A = get_adjacency_matrix(test_outer_diff, sigma)

In [ ]:
accs_3, nmis_3, aris_3 = [], [], []
for i in tqdm(range(10)):
    sc = SpectralClustering(n_clusters=k_est, affinity='precomputed', n_jobs=-1).fit(test_A.cpu().numpy())
    test_pred = sc.labels_
    
    accs_3.append(hungarian_accuracy(test_pred, y_test_unknown_save_codes))
    nmis_3.append(normalized_mutual_info_score(test_pred, y_test_unknown_save_codes))
    aris_3.append(adjusted_rand_score(test_pred, y_test_unknown_save_codes))

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_3)*100, np.std(accs_3)*100, np.mean(nmis_3)*100, np.std(nmis_3)*100, np.mean(aris_3)*100, np.std(aris_3)*100))

## 3.4 NCD Spectral Clustering <a class="anchor" id="3.4"></a>

In [ ]:
# We load the hyperparameters that were optimized through grid search
d = json.load(open("hyperparameters.json"))
ncdsc_params = d["NCD SC"]["Estimated k"][dataset_name]
print("Using hyperparameters:", ncdsc_params)

#### /!\ In this case, we estimate the number of clusters in the spectral embedding before the $k$-means /!\

In [ ]:
# (1) Get the spectral embedding for all the data (since the hyperparameters are adapted for all the data, not only the novel data)
x_test_full = torch.concat([x_test_known, x_test_unknown], axis=0)
full_spectral_embedding = get_spectral_embedding(x_test_full, ncdsc_params['n_components'], ncdsc_params['min_dist'])

In [ ]:
# (2) Estimate the number of clusters from the UNKNOWN DATA ONLY
unknown_spectral_embedding = full_spectral_embedding[len(x_test_known):]
ncd_sc_k_est, ncd_sc_silhouette_scores = estimate_k_with_silhouette(k_min=2, k_max=30, x=unknown_spectral_embedding)

plot_k_means_perf_curve(ncd_sc_silhouette_scores, np.arange(2, 30, 1), gt_k)

print(f"Estimated k={ncd_sc_k_est}, ground truth={gt_k} (difference={np.abs(gt_k - ncd_sc_k_est)})")

In [ ]:
# (3) With the estimations, run k-means on spectral embedding of novel data only
accs_4, nmis_4, aris_4 = [], [], []
for i in tqdm(range(5)):
    kmpp = k_means_pp(pre_centroids=None, k_new_centroids=len(np.unique(y_train_known)) + ncd_sc_k_est)
    kmpp.fit(full_spectral_embedding, tolerance=1e-10, n_iterations=1000, n_init=10)
    sil_y_test_full_pred = kmpp.predict_unknown_data(full_spectral_embedding).cpu().numpy()
    sil_y_test_unknown_pred = sil_y_test_full_pred[len(x_test_known):]
    
    accs_4.append(hungarian_accuracy(y_test_unknown_save_codes, sil_y_test_unknown_pred))
    nmis_4.append(normalized_mutual_info_score(y_test_unknown_save_codes, sil_y_test_unknown_pred))
    aris_4.append(adjusted_rand_score(y_test_unknown_save_codes, sil_y_test_unknown_pred))

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_4)*100, np.std(accs_4)*100, np.mean(nmis_4)*100, np.std(nmis_4)*100, np.mean(aris_4)*100, np.std(aris_4)*100))

# 4) NCD models <a class="anchor" id="4"></a>

## 4.1 Baseline <a class="anchor" id="4.1"></a>

In [ ]:
base_config = {
    'input_size': x_train.shape[1],
    'hidden_layers_dims': [math.floor(3*x_train.shape[1]/4), math.floor(2*x_train.shape[1]/4)],
    'activation_fct': 'relu',  # relu or sigmoid or tanh or None
    'use_batchnorm': True,  # True or False
    'use_norm': 'l2',  # None or 'l1' or 'l2'
    
    'n_classes': len(np.unique(y_train_known)),
    # 'n_clusters': ,  /!\ we will estimate this one in this notebook
    
    'clustering_model': 'kmeans',  # kmeans or ncd_kmeans or spectral_clustering or ncd_spectral_clustering
    'clustering_runs': 3,  # To compute the average accuracy of the clustering
    
    'batch_size': 512,
    'epochs': 200,
}

In [ ]:
# We load the hyperparameters that were optimized through grid search
d = json.load(open("hyperparameters.json"))
config = d["Baseline"]["Estimated k"][dataset_name]
print("Using hyperparameters:", config)

b_config = base_config.copy()
b_config.update(config)

#### /!\ In this case, we estimate the number of clusters in the latent space before the $k$-means /!\

In [ ]:
accs_5, nmis_5, aris_5 = [], [], []

for i in range(10):
    model = ArticleBaselineModel(b_config).to(device)
    
    # (1) Train the latent space
    losses_dict = model.train_on_known_classes(x_train_known=x_train_known, y_train_known=y_train_known,
                                               x_test_unknown=None, y_test_unknown=None, x_test_known=None, y_test_known=None,
                                               batch_size=b_config['batch_size'], lr=b_config['lr'], epochs=b_config['epochs'],
                                               n_clusters=None, clustering_runs=None, evaluate=False, disable_tqdm=True)
    model.eval()

    # (2) Estimate the number of clusters from the UNKNOWN DATA ONLY in the latent space
    with torch.no_grad():
        z_unknown = model.encoder_forward(x_train[y_train == unknown_class_value])
    
    baseline_k_est, baseline_silhouette_scores = estimate_k_with_silhouette(k_min=2, k_max=30, x=z_unknown)
    
    print(f"Estimated k={baseline_k_est}, ground truth={gt_k} (difference={np.abs(gt_k - baseline_k_est)})")

    # (3) In the latent space, run k-means with the estimated number of clusters
    accs, nmis, aris = [], [], []
    for i in range(5):
        y_test_unknown_pred = model.predict_new_data(baseline_k_est, x_test_unknown, x_test_known, y_test_known)
        accs.append(hungarian_accuracy(y_test_unknown_save_codes, y_test_unknown_pred))
        nmis.append(normalized_mutual_info_score(y_test_unknown_save_codes, y_test_unknown_pred))
        aris.append(adjusted_rand_score(y_test_unknown_save_codes, y_test_unknown_pred))
    
    accs_5.append(np.mean(accs))
    nmis_5.append(np.mean(nmis))
    aris_5.append(np.mean(aris))

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_5)*100, np.std(accs_5)*100, np.mean(nmis_5)*100, np.std(nmis_5)*100, np.mean(aris_5)*100, np.std(aris_5)*100))

## 4.2 PBN <a class="anchor" id="4.2"></a>

In [ ]:
base_config = {
    'input_size': x_train.shape[1],
    'hidden_layers_dims': [math.floor(3*x_train.shape[1]/4), math.floor(2*x_train.shape[1]/4)],
    'activation_fct': 'relu',  # relu or sigmoid or tanh or None
    'use_batchnorm': True,  # True or False
    'use_norm': 'l2',  # None or 'l1' or 'l2'
    
    'n_classes': len(np.unique(y_train_known)),
    # 'n_clusters': ?,  /!\ we will estimate this one in this notebook
    
    'clustering_model': 'kmeans',  # kmeans or ncd_kmeans or spectral_clustering or ncd_spectral_clustering
    'clustering_runs': 1,  # To compute the average accuracy of the clustering
    
    'batch_size': 512,
    'epochs': 200,
}

In [ ]:
# We load the hyperparameters that were optimized through grid search
d = json.load(open("hyperparameters.json"))
config = d["PBN"]["Estimated k"][dataset_name]
print("Using hyperparameters:", config)

pbn_config = base_config.copy()
pbn_config.update(config)

In [ ]:
accs_6, nmis_6, aris_6 = [], [], []

for i in range(10):
    model = PBNModel(pbn_config).to(device)

    # (1) Train the latent space
    losses_dict = model.train_on_known_classes(x_train=x_train, y_train=y_train, unknown_class_value=unknown_class_value, x_test_unknown=x_test_unknown, y_test_unknown=y_test_unknown_save_codes, x_test_known=x_test_known, y_test_known=y_test_known,
                                               batch_size=pbn_config['batch_size'], lr=pbn_config['lr'], epochs=pbn_config['epochs'], clustering_runs=pbn_config['clustering_runs'], w=pbn_config['w'],
                                               evaluate=False, disable_tqdm=True)
    model.eval()

    # (2) Estimate the number of clusters from the UNKNOWN DATA ONLY in the latent space
    with torch.no_grad():
        z_unknown = model.encoder_forward(x_train[y_train == unknown_class_value])
    
    baseline_k_est, baseline_silhouette_scores = estimate_k_with_silhouette(k_min=2, k_max=30, x=z_unknown)
    
    print(f"Estimated k={baseline_k_est}, ground truth={gt_k} (difference={np.abs(gt_k - baseline_k_est)})")

    # (3) In the latent space, run k-means with the estimated number of clusters
    accs, nmis, aris = [], [], []
    for i in range(5):
        y_test_unknown_pred = model.predict_new_data(baseline_k_est, x_test_unknown, x_test_known, y_test_known)
        accs.append(hungarian_accuracy(y_test_unknown_save_codes, y_test_unknown_pred))
        nmis.append(normalized_mutual_info_score(y_test_unknown_save_codes, y_test_unknown_pred))
        aris.append(adjusted_rand_score(y_test_unknown_save_codes, y_test_unknown_pred))
    
    accs_6.append(np.mean(accs))
    nmis_6.append(np.mean(nmis))
    aris_6.append(np.mean(aris))

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_6)*100, np.std(accs_6)*100, np.mean(nmis_6)*100, np.std(nmis_6)*100, np.mean(aris_6)*100, np.std(aris_6)*100))

## 4.3 TabularNCD <a class="anchor" id="4.3"></a>

In [ ]:
base_config = {
    'hidden_layers_dims': [math.floor(3*x_train.shape[1]/4), math.floor(2*x_train.shape[1]/4)],
    'input_size': x_train.shape[1],
    'n_known_classes': len(np.unique(y_train)),  # Takes into account the unknown class
    'activation_fct': 'relu',
    'use_batchnorm': True,
    'batch_size': 512,
    'epochs': 200,
    'M': 2000,
    
    'n_unknown_classes': k_est,  # /!\ We use the estimation here /!\
}

In [ ]:
# We load the hyperparameters that were optimized through grid search
d = json.load(open("hyperparameters.json"))
config = d["TabularNCD"]["Estimated k"][dataset_name]
print("Using hyperparameters:", config)

tncd_config = base_config.copy()
tncd_config.update(config)

In [ ]:
accs_7, nmis_7, aris_7 = [], [], []

for i in tqdm(range(10)):
    model = TabularNCDModel(tncd_config).to(device)
    losses_dict = model.joint_training(config=tncd_config,
                                       x_train=x_train, y_train=y_train,
                                       x_test_known=x_test_known, y_test_known=y_test_known,
                                       x_test_unknown=x_test_unknown, y_test_unknown=y_test_unknown_save_codes,
                                       y_train_unknown=y_train_unknown_save_codes,
                                       unknown_class_value=unknown_class_value,
                                       disable_tqdm=True)
    
    accs_7.append(losses_dict["test cluster ACC"][-1])
    nmis_7.append(losses_dict["test cluster NMI"][-1])
    aris_7.append(losses_dict["test cluster ARI"][-1])

In [ ]:
print("TEST: ACC={:.1f}±{:.1f} | NMI={:.1f}±{:.1f} | ARI={:.1f}±{:.1f}".format(np.mean(accs_7)*100, np.std(accs_7)*100, np.mean(nmis_7)*100, np.std(nmis_7)*100, np.mean(aris_7)*100, np.std(aris_7)*100))

In [ ]:
dataset_name